In [1]:
#Set environment variables for your notebook
import os 
import numpy as np
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/mpetkova/Dropbox/CREST/mariela_fish_credentials.json'

#Imports google cloud client library and initiates BQ service
from google.cloud import bigquery
bigquery_client = bigquery.Client()

# 
# Collects information from json states in a folder and a specific naming pattern
path2dir = '../';
names = os.listdir(path2dir);
cellid = list();
cellid_filename=list();
for ind in range(len(names)):
    content = names[ind].split('_')
    if ('state' in content):
        content[1]=content[1].replace(".json","")
        cellid.append(content[1])
        cellid_filename.append(names[ind])
        
#s save the skeletons with the same ids to the skeletons folder
import json
import pandas as pd 

for ind in range(len(cellid)):
    f = open(path2dir+cellid_filename[ind])
    data = json.load(f)
    
    # find the index of the segmentation layer
    seg_layer_idx = [];
    for i in range(len(data['layers'])):
        if data['layers'][i]['type']=='segmentation':
            seg_layer_idx = i
    # get the list of segment    
    base_seg_list = data['layers'][seg_layer_idx]['segments']
    base_seg_str = ', '.join(str(x) for x in base_seg_list)
    savename_and_path = "skeleton_"+cellid[ind]+".csv";

    if not os.path.exists(savename_and_path):
        print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
        df = pd.DataFrame()

        QUERY = """
        SELECT
            cast(skeletons.id as INT64) as seg_id, 
            nodes.position.x*2 as x,
            nodes.position.y*2 as y,
            nodes.position.z as z
        FROM
        `engert-goog-connectomics.raw_20211023_seg_20220521.skeletons` as skeletons,UNNEST(nodes) as nodes
        WHERE skeletons.id in {}
        """.format('('+base_seg_str+')')

        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])
        df.to_csv(savename_and_path, index=False)
print("done!")

working on:lafferent2...1/1cells
done!


In [ ]:
QUERY = f"""SELECT cast(skeletons.id as INT64) as seg_id, nodes.position.x*2 as x, nodes.position.y*2 as y, nodes.position.z as z FROM `engert-goog-connectomics.raw_20211023_seg_20220521.skeletons` as skeletons,UNNEST(nodes) as nodes WHERE skeletons.id in ({base_seg_str})"""

In [ ]:
query = f"""SELECT cast(skeletons.id as INT64), nodes.position.x*2 as x, nodes.position.y*2 as y, nodes.position.z as z FROM base_location as skeletons,UNNEST(nodes) as nodes WHERE seg_id IN ({q})"""